In [1]:
#-*- coding:utf-8
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

import GameEnv
import random
import numpy as np
import DQN

gameEnv = GameEnv.GlodenFlower([100000,100000])
RLModel = DQN.DQN(embedding_size=50,sequence_length=20,learning_rate=0.01,batch_size=1000)
gameEnv.debug = False
for episode in range(500000):
    # 初始化环境
    gameEnv.reset()

    playerI = gameEnv.getStartTurn()
    observation_this = [[],gameEnv.playerCards["A"],gameEnv.personStatus["A"]]
    if playerI == "B":
        availble_actions = gameEnv.chooseAvailbleAction(playerI)
        action,_ = RLModel.choose_action(np.array([gameEnv.transformObservationForB(observation_this)]),availble_actions,gameEnv.stepNum)
        #print (playerI, action, gameEnv.deskMoney, gameEnv.nowPrice)
        observation_next, reward, done = gameEnv.step(action, "B")
        playerI = "A"
        if done:
            continue

    while True:
        # DQN 根据观测值选择行为
        availble_actions = gameEnv.chooseAvailbleAction(playerI)
        action,_ = RLModel.choose_action(np.array([observation_this]),availble_actions,gameEnv.stepNum)
        # 环境根据行为给出下一个 state, reward, 是否终止
        #print (playerI, action, gameEnv.deskMoney, gameEnv.nowPrice,gameEnv.personStatus[playerI])
        observation_next, reward, done = gameEnv.stepA(action,RLModel)


        # DQN 存储记忆
        # RL.store_transition(observation, action, reward)
        RLModel.store_transition(observation_this, action, reward,done,observation_next,gameEnv.playerCards["B"],gameEnv.personStatus["A"],gameEnv.nowPrice)

        # 控制学习起始时间和频率 (先累积一些记忆再开始学习)
        if (episode > 10000):
            if episode % 10000 == 0 :print "episode",episode
            if episode % 10 == 0:
                RLModel.train(RLModel.experience_see_replay())
                RLModel.train_open(RLModel.experience_open_replay())

        # 将下一个 state_ 变为 下次循环的 state
        observation_this = observation_next

        # 如果终止, 就跳出循环
        if done:
            break

        # end of game
print('game over')



Instructions for updating:
Use `argmax` instead
('loss', 100, 259.82812)
('open loss', 100, 10.015593)
('loss', 200, 159.98347)
('open loss', 200, 3.6290262)
('loss', 300, 147.96391)
('open loss', 300, 3.5185876)
('loss', 400, 151.48378)
('open loss', 400, 0.81601524)
('loss', 500, 220.49083)
('open loss', 500, 1.141274)
('loss', 600, 225.09843)
('open loss', 600, 0.7413332)
('loss', 700, 177.30547)
('open loss', 700, 5.181875)
('loss', 800, 138.07266)
('open loss', 800, 1.5934232)
('loss', 900, 185.9479)
('open loss', 900, 3.5892887)
('loss', 1000, 146.33969)
('open loss', 1000, 0.7066427)
('loss', 1100, 176.68382)
('open loss', 1100, 2.7272987)
('loss', 1200, 189.12881)
('open loss', 1200, 1.6229471)
('loss', 1300, 141.0231)
('open loss', 1300, 1.9380943)
('loss', 1400, 1810.0466)
('open loss', 1400, 3.1920426)
episode 20000
('loss', 1500, 275.607)
('open loss', 1500, 3.037034)
('loss', 1600, 190.90926)
('open loss', 1600, 1.1163877)
('loss', 1700, 207.14317)
('open loss', 1700, 2.52

KeyboardInterrupt: 

# 验证终结点是否有效

In [2]:
import pandas as pd
l = RLModel.card_index_dicts.keys()
l.remove(" ")
card_list = []
input_data = []
for i in range(100000):
    cards = random.sample(l,3)
    card_list.append(cards)
    input_data.append([[],cards ,"看"])

In [3]:
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array(input_data)))

In [4]:
df = pd.DataFrame(data={"card":card_list,"score":res[:,8]})

import GameEnv
gameEnv = GameEnv.GlodenFlower([2000,2000])
d = {"豹子":10,"同花顺":9,"金花":8,"顺子":7,"对子":6,"单":5}
reverse_dict = {v: k for k, v in d.iteritems()}

def f1(x):
    return reverse_dict[gameEnv.score(x)]
    
df['card_level'] = df['card'].apply(lambda x:f1(x))

In [5]:
df.groupby("card_level").count()

,card,score
card_level,,
单,74764,74764
同花顺,220,220
对子,16798,16798
豹子,232,232
金花,5025,5025
顺子,2961,2961


In [6]:
df.groupby("card_level").mean()

,score
card_level,
单,-5.581727
同花顺,62.862709
对子,7.465661
豹子,11.503154
金花,8.099229
顺子,8.476376


In [7]:
df.sort_values("score",ascending=False).iloc[100:]
# df

,card,score,card_level
40328,"[spade_K, spade_1, heart_Q]",16.200001,顺子
40869,"[heart_Q, spade_1, spade_K]",16.200001,顺子
23969,"[heart_7, heart_1, heart_2]",16.177023,金花
81334,"[heart_7, heart_1, heart_2]",16.177023,金花
30457,"[heart_2, heart_1, heart_7]",16.177023,金花
22452,"[heart_2, heart_7, heart_1]",16.177023,金花
74130,"[heart_1, club_5, spade_1]",16.147739,对子
25479,"[heart_1, club_5, spade_1]",16.147739,对子
9855,"[heart_1, club_5, spade_1]",16.147739,对子
45411,"[club_5, spade_1, club_1]",16.088219,对子


# 单点检查


In [8]:
# RLModel.train(RLModel.exerience_replay_final_step())
res = RLModel.sess.run(RLModel.predictionsNotSee,feed_dict=RLModel._feed_dict(np.array([[[],  ['spade_6', 'spade_10', 'spade_8'],"闷"]])))
for i,v in  zip(RLModel.action_notsee_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_notsee_reverse_index_dicts[i],v

0 闷_2 1.1362231
1 闷_4 4.0263476
2 闷_8 1.6519315
3 闷开_0 -0.0860311
4 看 -2.184517


In [21]:
print res[0]
print RLModel._random_pick(["闷_2","闷_4","闷_8","闷开_0","看"],res[0])

[-11.574262    1.5971797  -3.8307111   0.3447262  -2.2230055]
看


In [20]:
# RLModel.train(RLModel.exerience_replay_final_step())
print RLModel.choose_action_max(np.array([[[],  ['heart_4', 'heart_2', 'spade_8'],"闷"]]),RLModel.actions_index_dicts.keys(),1,True)[0]

('\xe9\x97\xb7', '\xe9\x97\xb7')
('\xe9\x97\xb7_4', 0.24453352)
('\xe9\x97\xb7_2', 0.28051785)
('\xe9\x97\xb7\xe5\xbc\x80_0', 0.2544603)
('\xe9\x97\xb7_8', 0.22048834)
闷_2


In [12]:
# RLModel.train(RLModel.exerience_replay_final_step())
import tensorflow as tf
res = RLModel.sess.run(tf.equal(tf.arg_max(RLModel.predictionsNotSee,1),len(RLModel.action_notsee_index_dicts)),feed_dict=RLModel._feed_dict(np.array([[["B_看_20","A_闷_8","B_看_20"],  ['heart_4', 'heart_2', 'spade_8'],"闷"]])))
print res
#     

[False]


In [11]:
# RLModel.train(RLModel.exerience_replay_final_step())
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array([[["B_看_20","A_闷_8","B_看_20"],  ['heart_4', 'heart_2', 'heart_3'],"闷"]])))
for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_reverse_index_dicts[i],v

0 闷_2 -1.5942369
1 闷_4 6.0963144
2 闷_8 27.949627
3 闷开_0 3.9862342
4 看_2 0.0
5 看_5 0.0
6 看_10 0.0
7 看_20 0.0
8 开_0 0.0
9 丢_0 -0.0


# 检查完整的一把过程

In [26]:
def print_prob(status):
    res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(status))
    for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
        print i, RLModel.action_reverse_index_dicts[i],v
import GameEnv
gameEnv = GameEnv.GlodenFlower([20000,20000])
gameEnv.reset()
gameEnv.playerCards["A"] = ['club_2', 'heart_8', 'spade_10']
gameEnv.playerCards["B"] = ['club_3', 'heart_2', 'spade_9']
print gameEnv.playerCards["A"],gameEnv.playerCards["B"],"\n"
playerI = "A"
observation_this = [[],gameEnv.playerCards["A"],gameEnv.personStatus["A"]]
while True:
        # DQN 根据观测值选择行为
        availble_actions = gameEnv.chooseAvailbleAction(playerI)
        action,action_value = RLModel.choose_action_max(np.array([observation_this]),availble_actions,gameEnv.stepNum,True)
        print_prob(np.array([observation_this]))
        # 环境根据行为给出下一个 state, reward, 是否终止
        #print (playerI, action, gameEnv.deskMoney, gameEnv.nowPrice,gameEnv.personStatus[playerI])
        observation_next, reward, done = gameEnv.stepA(action,RLModel)

        # 将下一个 state_ 变为 下次循环的 state
        observation_this = observation_next
        print "______",playerI,action,action_value,"\n"
        # 如果终止, 就跳出循环
        if done:
            break
for i in observation_this[0]:
    print i

['club_2', 'heart_8', 'spade_10'] ['club_3', 'heart_2', 'spade_9'] 

('\xe9\x97\xb7', '\xe9\x97\xb7')
('\xe9\x97\xb7\xe5\xbc\x80_0', 0.2544603)
('\xe9\x97\xb7_4', 0.24453352)
('\xe9\x97\xb7_2', 0.28051785)
('\xe9\x97\xb7_8', 0.22048834)
0 闷_2 0.862558
1 闷_4 -0.51029044
2 闷_8 -1.5453668
3 闷开_0 -0.11236637
4 看_2 1.1037825
5 看_5 -2.4782772
6 看_10 -7.3136
7 看_20 -15.57754
8 开_0 -23.355452
9 丢_0 -0.97227585
______ A 闷_2 0.28051785 

('\xe9\x97\xb7', '\xe9\x97\xb7')
('\xe9\x97\xb7\xe5\xbc\x80_0', 0.2548142)
('\xe9\x97\xb7_4', 0.22328515)
('\xe9\x97\xb7_2', 0.3097111)
('\xe9\x97\xb7_8', 0.21218948)
0 闷_2 1.801272
1 闷_4 -1.4706305
2 闷_8 -1.9803306
3 闷开_0 -0.14978021
4 看_2 2.3119493
5 看_5 -1.6553155
6 看_10 -6.2624264
7 看_20 -21.73891
8 开_0 -24.083447
9 丢_0 -1.9734172
______ A 闷_2 0.3097111 

('\xe9\x97\xb7', '\xe9\x97\xb7')
('\xe9\x97\xb7\xe5\xbc\x80_0', 0.25183108)
('\xe9\x97\xb7_4', 0.64660376)
('\xe9\x97\xb7_8', 0.101565115)
0 闷_2 -8.072526
1 闷_4 11.872682
2 闷_8 -6.6376553
3 闷开_0 2.4429295
4

# 检查闷分布

In [9]:
for i in sorted(RLModel.memory_open.items(),key = lambda x:x[1],reverse = True):
    if len(i[0].split(",")) == 2 and i[0].split(",")[0] == "B_闷_2":
        print i[0],i[1],i[1][0] / float(i[1][1])

B_闷_2,A_闷_2 (4633.0, 1952) 2.37346311475
B_闷_2,A_闷_4 (3926.0, 1860) 2.11075268817
B_闷_2,A_闷_8 (226.0, 1242) 0.181964573269
B_闷_2,A_闷开_0 (-51.0, 1103) -0.0462375339982
B_闷_2,A_看_5 (-938.0, 380) -2.46842105263
B_闷_2,A_看_20 (-1864.0, 216) -8.62962962963
B_闷_2,A_看_10 (-2061.0, 241) -8.55186721992


In [23]:
for i in sorted(RLModel.memory_open.items(),key = lambda x:x[1],reverse = True):
    if len(i[0].split(",")) == 3 and i[0].split(",")[0] == "A_闷_2":
        print i[0],i[1],i[1][0] / float(i[1][1])

A_闷_2,B_看_20,A_闷_8 (1163.0, 71) 16.3802816901
A_闷_2,B_看_5,A_闷_4 (700.0, 47) 14.8936170213
A_闷_2,B_看_10,A_看_10 (687, 27) 25.4444444444
A_闷_2,B_看_10,A_闷_4 (359.0, 35) 10.2571428571
A_闷_2,B_闷_8,A_闷_8 (307.0, 123) 2.49593495935
A_闷_2,B_看_10,A_闷_8 (259.0, 29) 8.93103448276
A_闷_2,B_闷_4,A_闷_4 (148.0, 194) 0.762886597938
A_闷_2,B_看_20,A_闷开_0 (114.0, 42) 2.71428571429
A_闷_2,B_闷_2,A_闷_2 (99.0, 74) 1.33783783784
A_闷_2,B_看_5,A_闷_8 (64.0, 15) 4.26666666667
A_闷_2,B_看_10,A_闷开_0 (53.0, 31) 1.70967741935
A_闷_2,B_看_20,A_看_20 (37, 5) 7.4
A_闷_2,B_看_5,A_看_10 (29, 3) 9.66666666667
A_闷_2,B_看_5,A_闷_2 (-2.0, 14) -0.142857142857
A_闷_2,B_看_5,A_闷开_0 (-4.0, 14) -0.285714285714
A_闷_2,B_看_5,A_看_20 (-5, 4) -1.25
A_闷_2,B_看_5,A_看_5 (-10, 3) -3.33333333333
A_闷_2,B_闷_2,A_看_10 (-68.0, 14) -4.85714285714
A_闷_2,B_闷_8,A_看_20 (-83.0, 21) -3.95238095238
A_闷_2,B_闷_2,A_闷开_0 (-111.0, 83) -1.33734939759
A_闷_2,B_闷_4,A_看_10 (-112.0, 26) -4.30769230769
A_闷_2,B_闷_2,A_闷_8 (-113.0, 77) -1.46753246753
A_闷_2,B_闷_2,A_看_20 (-123.0, 15) -8.2


In [10]:
for i in sorted(RLModel.memory_open.items(),key = lambda x:x[1],reverse = True):
    if len(i[0].split(",")) == 1:
        print i[0],i[1],i[1][0] / float(i[1][1])

A_闷_2 (5190.0, 7418) 0.699649501213
A_闷_4 (3279.0, 11613) 0.282355980367
A_闷_8 (-548.0, 6332) -0.0865445356917
A_闷开_0 (-2728, 5351) -0.509811250234
A_看_5 (-4637.0, 972) -4.77057613169
A_看_10 (-5625.0, 805) -6.98757763975
A_看_2 (-6773.0, 1698) -3.98881036514
A_看_20 (-7145.0, 809) -8.83189122373


In [2]:
RLModel.experience_open_replay()

[['A_\xe9\x97\xb7_4,B_\xe9\x97\xb7_8', '\xe9\x97\xb7_8', 7.8],
 ['B_\xe9\x97\xb7_8', '\xe9\x97\xb7\xe5\xbc\x80_0', 0.47368421052631576],
 ['A_\xe9\x97\xb7_4,B_\xe9\x97\xb7_8,A_\xe9\x97\xb7_8,B_\xe9\x97\xb7_8',
  '\xe9\x97\xb7_8',
  17.0],
 ['B_\xe9\x97\xb7_2', '\xe9\x97\xb7_2', 2.037037037037037],
 ['A_\xe9\x97\xb7_2,B_\xe9\x97\xb7_8', '\xe9\x97\xb7_8', 1.0],
 ['B_\xe9\x97\xb7_2,A_\xe9\x97\xb7_4,B_\xe7\x9c\x8b_10', '', -1.0],
 ['B_\xe7\x9c\x8b_5,A_\xe9\x97\xb7_4,B_\xe7\x9c\x8b_20', '', 10.5],
 ['A_\xe9\x97\xb7_4,B_\xe9\x97\xb7_8', '', -3.0],
 ['B_\xe9\x97\xb7_2', '\xe9\x97\xb7\xe5\xbc\x80_0', -1.105263157894737],
 ['B_\xe9\x97\xb7_8,A_\xe9\x97\xb7_8,B_\xe7\x9c\x8b_20', '', 10.0],
 ['A_\xe9\x97\xb7_4,B_\xe9\x97\xb7_4', '\xe9\x97\xb7_4', 14.6],
 ['B_\xe9\x97\xb7_2,A_\xe9\x97\xb7_8,B_\xe9\x97\xb7_8',
  '\xe9\x97\xb7\xe5\xbc\x80_0',
  -17.0],
 ['A_\xe9\x97\xb7_8,B_\xe7\x9c\x8b_20',
  '\xe9\x97\xb7\xe5\xbc\x80_0',
  0.7333333333333333],
 ['A_\xe9\x97\xb7_2,B_\xe7\x9c\x8b_5', '\xe9\x97\xb7_4

In [27]:
train_data = [["","闷_2",0.34]]
playSequenceStr = [i[0].split(",") if i[0] != "" else [] for i in train_data]
playSequenceIndex = [[RLModel.seq_action_index_dicts[i] if i != "" else len(RLModel.seq_action_index_dicts) for i in j] + [len(RLModel.seq_action_index_dicts)] * (len(RLModel.seq_action_index_dicts) - len(j)) for j in playSequenceStr]
playSequenceLength = [len(i) + 1 for i in playSequenceStr]
actionIndex = [RLModel.action_notsee_index_dicts[i[1]] for i in train_data]
yInput = [i[2] for i in train_data]
feed_dict = {RLModel.playSequenceInput: np.array(playSequenceIndex),
             RLModel.actionInputOpen: RLModel._one_hot(actionIndex,len(RLModel.action_notsee_index_dicts)),RLModel.playSequenceLengthInput: np.array(playSequenceLength),
             RLModel.yInputOpen: np.array(yInput)}
# for i in range(100):
rs, global_step, loss = RLModel.sess.run([RLModel.predictionsNotSee, RLModel.global_steps_open, RLModel.loss_open], feed_dict=feed_dict)

print loss

1.0397981


In [29]:
# RLModel.train(RLModel.exerience_replay_final_step())
res = RLModel.sess.run(RLModel.predictionsNotSee,feed_dict=RLModel._feed_dict(np.array([[[],  ['heart_4', 'heart_2', 'spade_8'],"闷"]])))
for i,v in  zip(RLModel.action_notsee_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_notsee_reverse_index_dicts[i],v

0 闷_2 0.49433714
1 闷_4 5.0349956
2 闷_8 1.5542467
3 闷开_0 -0.15048584
4 看 -4.3465657


In [12]:

# RLModel.train(RLModel.exerience_replay_final_step())
res = RLModel.sess.run(RLModel.predictionsNotSee,feed_dict=RLModel._feed_dict(np.array([[[],  ['heart_4', 'heart_2', 'spade_8'],"闷"]])))
for i,v in  zip(RLModel.action_notsee_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_notsee_reverse_index_dicts[i],v

0 闷_2 11.790662
1 闷_4 11.684034
2 闷_8 10.927329
3 闷开_0 1.2110952
4 看 -3.9434104
